In [1]:
import numpy as np
import seaborn as sb
import pandas as pd
import matplotlib.pyplot as plt
from pandas import Grouper
from pandas import DataFrame
import csv
import pickle
import math
import matrixprofile as mp
from matplotlib.patches import Rectangle
from fbprophet import Prophet
from prophet.plot import plot_yearly
from sklearn.metrics import mean_squared_error
from prophet.diagnostics import cross_validation
from prophet.diagnostics import performance_metrics
from sklearn.model_selection import TimeSeriesSplit
from numpy import sqrt
import datetime
from datetime import timedelta
import plotly.graph_objs as go
from sklearn.metrics import r2_score

/home/jonathan/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
DataMeteo = pd.read_csv("/home/jonathan/tesis/Data_Temperatura_Lluvia/may_meteo20.csv", sep=';')
DataMeteo.head(1)

,PROVINCIA,MUNICIPIO,ESTACION,MAGNITUD,PUNTO_MUESTREO,ANO,MES,DIA,H01,V01,...,H20,V20,H21,V21,H22,V22,H23,V23,H24,V24
0,28,79,102,81,28079102_81_98,2020,5,1,2.3,V,...,3.82,V,2.58,V,1.8,V,1.17,V,1.47,V


# Obtenemos datos de temperatura de la estación Chamberí
### Punto de muestreo de Chamartín 28079111, estación 111 
### Punto de muestreo de Centro 28079110, estación 110
### Punto de muestreo de Chamberí 28079109, estación 109
### Debido a que no se encuentra la estación 111 se procede a utilizar los datos de la estación Chamberí

In [3]:
DataFiltrar = DataMeteo.loc[DataMeteo.loc[:,'ESTACION'] == 109]

In [4]:
DataTemp = DataFiltrar.loc[DataFiltrar.loc[:,'MAGNITUD'] == 83]

In [5]:
DataTemp.head(1)

,PROVINCIA,MUNICIPIO,ESTACION,MAGNITUD,PUNTO_MUESTREO,ANO,MES,DIA,H01,V01,...,H20,V20,H21,V21,H22,V22,H23,V23,H24,V24
1147,28,79,109,83,28079109_83_98,2020,5,1,13.4,V,...,20.3,V,18.8,V,17.7,V,17.0,V,16.4,V


# Obtenemos datos de Precipitación de la estación Hortaleza 
## Las estaciones que tienen datos de precipitación son las siguientes: 102,103,106,107,108,24,39,54,56,59

In [6]:
DataFiltrar2 = DataMeteo.loc[DataMeteo.loc[:,'ESTACION'] == 107]
DataPrec = DataFiltrar2.loc[DataFiltrar2.loc[:,'MAGNITUD'] == 89]
DataPrec.head(1)

,PROVINCIA,MUNICIPIO,ESTACION,MAGNITUD,PUNTO_MUESTREO,ANO,MES,DIA,H01,V01,...,H20,V20,H21,V21,H22,V22,H23,V23,H24,V24
899,28,79,107,89,28079107_89_98,2020,5,1,0.0,V,...,0.0,V,0.0,V,0.0,V,0.0,V,0.0,V


In [7]:
DataTemp = DataTemp.drop(columns=['PROVINCIA','MUNICIPIO','ESTACION','MAGNITUD','V01','V02','V03','V04','V05','V06','V07','V08','V09','V10','V11','V12','V13','V14','V15','V16','V17','V18','V19','V20','V21','V22','V23','V24'])
DataPrec = DataPrec.drop(columns=['PROVINCIA','MUNICIPIO','ESTACION','MAGNITUD','V01','V02','V03','V04','V05','V06','V07','V08','V09','V10','V11','V12','V13','V14','V15','V16','V17','V18','V19','V20','V21','V22','V23','V24'])
print(DataTemp.head(1))
print(DataPrec.head(1))

      PUNTO_MUESTREO   ANO  MES  DIA   H01   H02   H03   H04   H05   H06  ...  \
1147  28079109_83_98  2020    5    1  13.4  13.0  12.7  12.2  11.8  11.7  ...   

       H15   H16   H17   H18   H19   H20   H21   H22   H23   H24  
1147  20.5  21.5  22.3  23.1  22.3  20.3  18.8  17.7  17.0  16.4  

[1 rows x 28 columns]
     PUNTO_MUESTREO   ANO  MES  DIA  H01  H02  H03  H04  H05  H06  ...  H15  \
899  28079107_89_98  2020    5    1  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0   

     H16  H17  H18  H19  H20  H21  H22  H23  H24  
899  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  

[1 rows x 28 columns]
